# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,24.99,78,40,6.17,CC,1696115274
1,1,bredasdorp,-34.5322,20.0403,10.65,74,18,1.67,ZA,1696115274
2,2,ad dabbah,18.0500,30.9500,34.29,14,22,2.81,SD,1696115275
3,3,vorgashor,67.5833,63.9500,3.80,98,100,6.61,RU,1696115275
4,4,yellowknife,62.4560,-114.3525,8.56,85,36,3.58,CA,1696115275


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',  # Use humidity as color
    s='Humidity',  # Use humidity as size
    cmap='Blues',       # Specify the color map
    colorbar=True,      # Show the color bar
    title='City Humidity Map',
    width=800,
    height=600
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values


# Define the conditions for filtering
temperature_condition = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)
wind_speed_condition = city_data_df['Wind Speed'] < 4.5
cloudiness_condition = city_data_df['Cloudiness'] == 0

# Combine the conditions using the '&' operator
ideal_weather_condition = temperature_condition & wind_speed_condition & cloudiness_condition

# Use the boolean mask to filter the DataFrame
ideal_weather_cities = city_data_df[ideal_weather_condition]

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,keren,15.7778,38.4581,22.02,44,0,2.72,ER,1696115280
31,31,lincheng,19.9078,109.6858,25.13,88,0,1.84,CN,1696115281
37,37,sidi akkacha,36.4647,1.3026,22.81,64,0,2.98,DZ,1696115282
107,107,arar,30.9753,41.0381,25.30,25,0,3.34,SA,1696115298
119,119,rakops,-21.0167,24.3333,23.91,27,0,1.31,BW,1696115301
155,155,douz,33.4663,9.0203,21.91,66,0,3.88,TN,1696115310
218,218,alcala de xivert,40.2580,0.2689,25.68,59,0,3.90,ES,1696115325
226,226,chubbuck,42.9208,-112.4661,21.02,51,0,3.09,US,1696115326
239,239,deltebre,40.7499,0.7868,23.64,71,0,2.40,ES,1696115329
299,299,iranshahr,27.2025,60.6848,25.40,16,0,1.42,IR,1696115344


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Create a new DataFrame 'hotel_df' by copying selected columns from 'city_data_df'
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,78,
1,bredasdorp,ZA,-34.5322,20.0403,74,
2,ad dabbah,SD,18.0500,30.9500,14,
3,vorgashor,RU,67.5833,63.9500,98,
4,yellowknife,CA,62.4560,-114.3525,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "api_key": geoapify_key,
    "radius": radius,
    "categories" :"accommodation.hotel"
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
   
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        #hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df





Starting hotel search
west island - nearest hotel: Cocos Village Bungalows
bredasdorp - nearest hotel: Victoria Hotel
ad dabbah - nearest hotel: No hotel found
vorgashor - nearest hotel: София
yellowknife - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
port-aux-francais - nearest hotel: Keravel
edinburgh of the seven seas - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
port elizabeth - nearest hotel: Waterford Hotel
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
solt - nearest hotel: No hotel found
kodinsk - nearest hotel: Ангара
blackmans bay - nearest hotel: Villa Howden
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
olonkinbyen - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
colchani - nearest hotel: Hotel de Sal - Sol de Mañana
tiksi - nearest hotel: Арктика
new norfolk - nearest hotel: Woodbridge on the Derwent

tateyama - nearest hotel: No hotel found
riyadh - nearest hotel: White Palace Hotel
sechura - nearest hotel: Hospedaje Claritza
mocoa - nearest hotel: Alejandro I
san julian - nearest hotel: No hotel found
melfort - nearest hotel: No hotel found
chonchi - nearest hotel: Hotel Huildín
smithers - nearest hotel: Sunshine Inn Hotel
namibe - nearest hotel: Hotel Chik Chik Namibe
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
gubkinskiy - nearest hotel: Сибирь
san juan de miraflores - nearest hotel: Venue Hospedaje
stanley - nearest hotel: Hotel 52
al mishkhab - nearest hotel: خال رسول
fale old settlement - nearest hotel: No hotel found
lesosibirsk - nearest hotel: No hotel found
ubari - nearest hotel: فندق اوباري
hermanus - nearest hotel: Aloe guest house
dawesville - nearest hotel: Baymont by Wyndham Thomasville
olinda - nearest hotel: Fidji Motel
el alto - nearest hotel: No hotel found
yuzhno-kurilsk - nearest hotel: Dom Druzhby
jilma - nearest hotel: No hotel foun

gadzhiyevo - nearest hotel: Оазис
borogontsy - nearest hotel: No hotel found
suntar - nearest hotel: No hotel found
wewak - nearest hotel: Village Inn
ivdel' - nearest hotel: гостиница Ивдель
khandyga - nearest hotel: Апельсин
san fernando de apure - nearest hotel: Hotel Plaza
umm kaddadah - nearest hotel: No hotel found
n'dalatando - nearest hotel: Hotel Terminus
extrema - nearest hotel: Hotel San Marco
darwin - nearest hotel: Mantra Pantanas Darwin
saudarkrokur - nearest hotel: Mikligarður
aomori - nearest hotel: Art hotel colors
pelotas - nearest hotel: Hotel Santiago
erdenet - nearest hotel: No hotel found
moree - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
harper - nearest hotel: No hotel found
manokwari - nearest hotel: Hotel Arfak
weiser - nearest hotel: No hotel found
ust'-ilimsk - nearest hotel: Усть-Илимск
bilokurakyne - nearest hotel: Палуба
palopo - nearest hotel: Hotel Mulia Indah
kindu - nearest hotel: No hotel found
queenstown - nearest hotel:

levuka - nearest hotel: No hotel found
island harbour - nearest hotel: Premier Inn
whitecourt - nearest hotel: Microtel Inn & Suites by Wyndham Whitecourt
shumikha - nearest hotel: No hotel found
urumqi - nearest hotel: 百时快捷酒店红山店
gaalkacyo - nearest hotel: Citycenter وسط المدينة
soe - nearest hotel: DENA
afaahiti - nearest hotel: Omati Lodge
touros - nearest hotel: Pousada Atlântico
savanna-la-mar - nearest hotel: Kibo Hotel
sisimiut - nearest hotel: Sømandshjemmet
carutapera - nearest hotel: No hotel found
lagoa - nearest hotel: Casa da Madrinha
fort william - nearest hotel: Highland Hotel
markovo - nearest hotel: No hotel found
kampong speu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,78,Cocos Village Bungalows
1,bredasdorp,ZA,-34.5322,20.0403,74,Victoria Hotel
2,ad dabbah,SD,18.0500,30.9500,14,No hotel found
3,vorgashor,RU,67.5833,63.9500,98,София
4,yellowknife,CA,62.4560,-114.3525,85,No hotel found
...,...,...,...,...,...,...
583,carutapera,BR,-1.1950,-46.0200,77,No hotel found
584,lagoa,PT,39.0500,-27.9833,69,Casa da Madrinha
585,fort william,GB,56.8165,-5.1121,94,Highland Hotel
586,markovo,RU,57.3206,107.0881,96,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    s='Humidity',
    cmap='Blues',
    colorbar=True,
    title='City Humidity Map',
    width=800,
    height=600,
    hover_cols=['Hotel Name', 'Country', 'Humidity (%)']  # Customize the hover tooltip
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,Country)